# Guide to Feedback Control

In the previous notebook, we learned how to set up a static simulation. Now, we'll explore one of the most powerful features of the Smart Water Platform: building dynamic, closed-loop control systems using **Agents**.

A feedback control system works on a simple but powerful principle: **Sense, Decide, Act**. We will build a system that:
1.  **Senses** the water level in a reservoir.
2.  **Decides** how to adjust a gate to bring the water level to a desired setpoint.
3.  **Acts** on the gate to change its opening.

## The MessageBus: The Central Nervous System

The key to decoupling our components is the `MessageBus`. It allows different parts of the system to communicate without needing to know about each other directly. This is achieved through a **publish-subscribe** pattern:

- An agent can **publish** a message (e.g., a piece of data like a water level) to a named channel, called a **topic**.
- Any other agent can **subscribe** to that topic to receive all messages published to it.

This is how our control agent will get information about the reservoir's state.

## The Cast of Agents

To build our control loop, we need a few specialized agents:

### 1. The Sensor: `DigitalTwinAgent`
The `DigitalTwinAgent` is our sensor. Its job is to read the state of a physical component (its "digital twin") and publish it to a topic on the message bus. In our case, it will read the `water_level` from our reservoir and publish it to the `"state.reservoir.level"` topic.

### 2. The Decision-Maker: `PIDController`
The `PIDController` (Proportional-Integral-Derivative) is a classic control algorithm. It takes a `setpoint` (the desired value) and a `process_variable` (the current measured value) and calculates an appropriate control action to minimize the error between them. It doesn't know *what* it's controlling, only that it needs to reduce the error.

### 3. The Brain: `LocalControlAgent`
The `LocalControlAgent` (LCA) is the brain of our operation. It connects the sensor and the decision-maker to an actuator. The LCA is configured to:
- **Subscribe** to a state topic (e.g., `"state.reservoir.level"`).
- When it receives a message, it passes the value to its internal `PIDController`.
- It takes the output from the PID controller and **publishes** it as a control signal to an action topic (e.g., `"action.gate.opening"`).

### 4. The Actuator: A Message-Aware `Gate`
Finally, we need an actuator that can receive the command from the LCA. We configure our `Gate` to subscribe to the `"action.gate.opening"` topic on the message bus. When it receives a message, it updates its target opening, completing the control loop.

## Visualizing the Architecture

The diagram below shows this architecture. Ovals represent agents, and rectangles represent physical components. The arrows show the flow of information through the `MessageBus`.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

G = nx.DiGraph()

# Add nodes with type attribute
G.add_node("Reservoir", type='physical')
G.add_node("Gate", type='physical')
G.add_node("Twin Agent", type='agent')
G.add_node("LCA (PID)", type='agent')

# Add edges representing information flow
G.add_edge("Reservoir", "Twin Agent", label='reads state')
G.add_edge("Twin Agent", "LCA (PID)", label='publishes state\n(state.reservoir.level)')
G.add_edge("LCA (PID)", "Gate", label='publishes action\n(action.gate.opening)')

node_colors = ['lightblue' if G.nodes[n]['type'] == 'physical' else 'lightgreen' for n in G.nodes]
pos = nx.circular_layout(G)

plt.figure(figsize=(10, 8))
nx.draw(G, pos, with_labels=True, node_size=5000, node_color=node_colors, font_size=10, font_weight='bold', arrowsize=20)
edge_labels = nx.get_edge_attributes(G, 'label')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red')
plt.title('Feedback Control Architecture')
plt.show()

## The Simulation Code
Now let's see the code that builds and runs this system.

In [ ]:
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.simulation_identification.physical_objects.gate import Gate
from swp.local_agents.control.pid_controller import PIDController
from swp.local_agents.control.local_control_agent import LocalControlAgent
from swp.local_agents.perception.digital_twin_agent import DigitalTwinAgent
from swp.core_engine.testing.simulation_harness import SimulationHarness
import pandas as pd

# 1. Setup Harness and MessageBus
simulation_config = {'duration': 300, 'dt': 1.0}
harness = SimulationHarness(config=simulation_config)
message_bus = harness.message_bus

# 2. Define Communication Topics
RESERVOIR_STATE_TOPIC = "state.reservoir.level"
GATE_ACTION_TOPIC = "action.gate.opening"

# 3. Create Physical and Agent Components
reservoir = Reservoir(name="reservoir_1", initial_state={'volume': 21e6, 'water_level': 14.0}, parameters={'surface_area': 1.5e6})
gate = Gate(name="gate_1", initial_state={'opening': 0.1}, parameters={'width': 10}, message_bus=message_bus, action_topic=GATE_ACTION_TOPIC)

twin_agent = DigitalTwinAgent(agent_id="twin_agent_reservoir_1", simulated_object=reservoir, message_bus=message_bus, state_topic=RESERVOIR_STATE_TOPIC)
pid_controller = PIDController(Kp=-0.5, Ki=-0.01, Kd=-0.1, setpoint=12.0)
control_agent = LocalControlAgent(agent_id="control_agent_gate_1", controller=pid_controller, message_bus=message_bus, observation_topic=RESERVOIR_STATE_TOPIC, observation_key='water_level', action_topic=GATE_ACTION_TOPIC, dt=harness.dt)

# 4. Add everything to the harness
harness.add_component(reservoir)
harness.add_component(gate)
harness.add_agent(twin_agent)
harness.add_agent(control_agent)
harness.add_connection("reservoir_1", "gate_1")

# 5. Build and run the Multi-Agent System (MAS) simulation
import sys
original_stdout = sys.stdout
with open('simulation_log.txt', 'w') as f:
    sys.stdout = f
    harness.build()
    harness.run_mas_simulation()
sys.stdout = original_stdout

print("Feedback control simulation complete.")

## Results and Visualization

The plots below show the results. The water level starts at 14.0m and is driven by the controller towards the 12.0m setpoint. We can see the gate opening adjusted by the LCA to achieve this.

In [ ]:
time = [h['time'] for h in harness.history]
res_level = [h['reservoir_1']['water_level'] for h in harness.history]
gate_opening = [h['gate_1']['opening'] for h in harness.history]

df = pd.DataFrame({'Time': time, 'Reservoir Level': res_level, 'Gate Opening': gate_opening})

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

ax1.plot(df['Time'], df['Reservoir Level'], label='Reservoir Water Level')
ax1.axhline(y=12.0, color='r', linestyle='--', label='Setpoint (12.0m)')
ax1.set_ylabel('Water Level (m)')
ax1.set_title('Feedback Control Performance')
ax1.grid(True)
ax1.legend()

ax2.plot(df['Time'], df['Gate Opening'], label='Gate Opening', color='purple')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Gate Opening (%)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()